<a href="https://colab.research.google.com/github/dangdang2222/LG_Aimers/blob/main/finalmodel_featureselectiony08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2개로 가기(xgboost, randomforest)
1.이상치 제거
-->1% 클리핑(원래는 각 X 피쳐 속성 다 고려해야하지만 시간이 없으니깐...)
2. 인희님이 feature 만드신거 의미있어보이므로 일단은 feature 만들어놓고 생각
--> feature 만드는건 바로 아래 글 참조
feature 추가해서 만들고, 해당 feature 만드는데 사용되었던 feature은 일단 drop x,
+반드시 통과여부 drop하기******
3. rough하게 준 모델 parameter tuning for each y feature
xgboost and randomforest
xgboost initial 파라미터
xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
randomforest initial 파라미터
RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
)

In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [106]:
import pandas as pd
import random
import os
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [107]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,13): # ignore 'ID'
        rmse = mean_squared_error(np.array(gt)[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(np.array(gt)[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [108]:
def lg_nrmse_one(gt, preds):
    # Y_08의 경우는 가중치 1.2 부여. 8 이후부터는 모두 가중치 1.0
    all_nrmse = []
    rmse = mean_squared_error(np.array(gt), preds, squared=False)
    nrmse = rmse/np.mean(np.abs(np.array(gt)))
    all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse)
    return score

#얘를 쓸것!!!

In [109]:
train_df = pd.read_csv('/content/drive/MyDrive/LG Aimer/train.csv').drop(columns="ID")
train_x = train_df.filter(regex='X') # Input : X Featrue
train_x=train_x.drop(columns=['X_04','X_23','X_47','X_48'])
train_y = train_df.filter(regex='Y') # Output : Y Feature



#Y_08

In [110]:
train_y08 = train_y['Y_08']
train_y08

0       -25.381
1       -26.619
2       -26.238
3       -25.426
4       -25.376
          ...  
39602   -26.486
39603   -27.308
39604   -26.502
39605   -26.760
39606   -26.054
Name: Y_08, Length: 39607, dtype: float64

In [111]:
train_y08=pd.DataFrame(train_y08, index= train_y.index)

#클리핑

In [112]:
from sklearn.model_selection import train_test_split

# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x.quantile(0.01)
p99 = train_x.quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x = train_x.clip(p01, p99, axis=1)


#핀 휜 정도 피쳐 만들기

In [113]:
# 커넥터 핀 휨 
train_x['X__21'] = (train_x['X_24'] - train_x['X_12'])
train_x['X__22'] = (train_x['X_25'] - train_x['X_12'])
train_x['X__23'] = (train_x['X_26'] - train_x['X_12'])
train_x['X__24'] = (train_x['X_27'] - train_x['X_12'])
train_x['X__25'] = (train_x['X_28'] - train_x['X_12'])
train_x['X__26'] = (train_x['X_29'] - train_x['X_12'])
#이후, RFE 상에서 혹여나 새로운 변수를 만드는데 기여한 기존 변수들(12, 24,25,26,27,28,29)가 important하게 잡힐 수 있으므로 일단은 놔둡니다.
#또한, 현재 train_x 상에서는 통과여부 변수들은 모두 drop된 상태입니다.


In [94]:
train_x.columns

Index(['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56', 'X__21',
       'X__22', 'X__23', 'X__24', 'X__25', 'X__26'],
      dtype='object')

RFE 돌리기

#XGBoost

In [95]:
import xgboost as xgb
regressor1= xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,
                           colsample_bytree=1, max_depth=7)


In [97]:
from sklearn.feature_selection import RFE
n_features_to_select=1
rfe1=RFE(regressor1, n_features_to_select=n_features_to_select)

rfe1.fit(train_x, train_y08) 
#다른 데에서 RFE fit할때는 전체 데이터로 하던거 같더라구요
from operator import itemgetter
features = train_x.columns.to_list()
rank_xgb = []
for x,y in (sorted(zip(rfe1.ranking_,features), key=itemgetter(0))):
  print(x,y)
  rank_xgb.append(y)



[17:58:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:59:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:01:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:02:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:02:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:03:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

#이후 최적의 경우가 어떨때인지를 본다

In [98]:
score_list=[]
for i in range(len(rank_xgb)):
    cols_forxgb = rank_xgb[:(i+1)]
    train_x_xgb = train_x[cols_forxgb]
    x_train,x_valid, y_train,y_valid = train_test_split(train_x_xgb,train_y08,test_size=0.3) 

    model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, gamma=1, subsample=0.75,colsample_bytree=1, max_depth=7)
    model_xgb.fit(x_train, y_train) ##xgboost error....?
    y_predict = model_xgb.predict(x_valid)
    score = lg_nrmse_one(y_valid, y_predict)
    score_list.append(score)

#여기서 최대인 애 인덱스 찾으면 해당 경우가 최적


[18:21:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:21:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:21:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:21:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:21:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:21:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:21:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:21:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [100]:
score_list

[0.02975321097143936,
 0.029504974890413333,
 0.0292903535932211,
 0.029331331377593528,
 0.029502159690002826,
 0.028604390170253,
 0.02861692775562208,
 0.028840988887590704,
 0.028684553704314775,
 0.02874963817754305,
 0.028658038530014945,
 0.02909004345468198,
 0.028848218811025037,
 0.028355498461554246,
 0.0284310729492112,
 0.02883997577870389,
 0.02859822498828044,
 0.02852944816345522,
 0.02874150729209262,
 0.028668085512722246,
 0.028648253734709223,
 0.02860907635617029,
 0.028615194846843595,
 0.02815771422052461,
 0.028756393877209324,
 0.028719460111809838,
 0.0284722906708535,
 0.028800734805191026,
 0.028807604298047268,
 0.02857131263606098,
 0.028610466172156436,
 0.028588141440752748,
 0.028625437302329852,
 0.0284989265041105,
 0.02842428607483505,
 0.028665676778561095,
 0.02874785596416284,
 0.028532092850826585,
 0.028669022716100886,
 0.028948525135239232,
 0.02875240706741273,
 0.028647012161325362,
 0.02873660364892133,
 0.028492463971083064,
 0.02889327455

In [102]:
tmp = min(score_list)
index = score_list.index(tmp)

print(index)

23


#Randomforest

In [114]:
regressor2= RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
from sklearn.feature_selection import RFE
n_features_to_select=1
rfe2=RFE(regressor2, n_features_to_select=n_features_to_select)

rfe2.fit(train_x, train_y08)
from operator import itemgetter
features = train_x.columns.to_list()
rank_rf = []
for x,y in (sorted(zip(rfe2.ranking_,features), key=itemgetter(0))):
  print(x,y)
  rank_rf.append(y)



/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expe

1 X_49
2 X_08
3 X_09
4 X_07
5 X_56
6 X_03
7 X_51
8 X_54
9 X_55
10 X_53
11 X_05
12 X_52
13 X_20
14 X_50
15 X_39
16 X_21
17 X_22
18 X_19
19 X_32
20 X_45
21 X_38
22 X_36
23 X__26
24 X_35
25 X_37
26 X_34
27 X_16
28 X_33
29 X_43
30 X__25
31 X_42
32 X_31
33 X_40
34 X_25
35 X_15
36 X_44
37 X_13
38 X_30
39 X_41
40 X__24
41 X_01
42 X__23
43 X_28
44 X_12
45 X_14
46 X_27
47 X__21
48 X_06
49 X_29
50 X_17
51 X_26
52 X__22
53 X_18
54 X_24
55 X_46
56 X_02
57 X_11
58 X_10


In [ ]:
score_list=[]
for i in range(len(rank_rf)):
  cols_forrf = rank_rf[:(i+1)]
  train_x_rf = train_x[cols_forrf]
  x_train,x_valid, y_train,y_valid = train_test_split(train_x_rf,train_y08,test_size=0.3) 
  model_rf =RandomForestRegressor(criterion="squared_error", random_state=2022, n_jobs=-1, n_estimators=300, max_depth=150, min_samples_leaf=2, min_samples_split=2)
  model_rf.fit(x_train,y_train)
  y_predict = model_rf.predict(x_valid)
  score = lg_nrmse_one(y_valid, y_predict)
  score_list.append(score)

#여기서 최대인 애 인덱스 찾으면 해당 경우가 최적


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConv

In [ ]:
tmp = max(score_list)
index = score_list.index(tmp)
print(index)

여기까지가 feature selection part!
추가로 parameter tuning 후 최종적인 모델 build!